# Problem set 2

Network models of linear and integer programming.

<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#Chapter-8---Network-Models" data-toc-modified-id="Chapter-8---Network-Models-1">Chapter 8 - Network Models</a></span><ul class="toc-item"><li><span><a href="#Exercise-7" data-toc-modified-id="Exercise-7-1.1">Exercise 7</a></span></li></ul></li><li><span><a href="#Reference" data-toc-modified-id="Reference-2">Reference</a></span></li></ul></div>

In [ ]:
using Cbc, JuMP

## Chapter 8 - Network Models

### Exercise 7

An automobile association is organizing a series of car races that will last for four days.
The organizers know that $r_j \ge 0$ special tires in good condition will be required on each of the four successive days, $j = 1, 2, 3, 4$.
They can meet these needs either by buying new tires at $P$ dollars apiece or by reshaping used tires (reshaping is a technique by which the grooves on the tire are deepened, using a special profile-shaped tool).
Two kinds of service are available for reshaping:
normal service, which takes one full day at $N$ dollars a tire, and quick service, which takes overnight at $Q$ dollars a tire.
How should the association, which starts out with no special tires, meet the daily requirements at minimal cost?

**(a)** Formulate a mathematical model for the above problem.
Does it exhibit the characteristics of a network problem?
Why?
(Hint.
Take into account the fact that, at the end of day $j$, some used tires may not be sent to reshaping.)

**Solução:**

The formulation as a network model is as follows.

<img src="../images/mit_problem_7.png" alt="Drawing" style="width: 600px;"/>

There are a node for the tires store, one node for each day and also one node for each day stock.

The stock can send tires to each day, with no restriction in number of tires sold and also no limits to transportation to it day, but each tire costs $P$.

All used tires in one day must be sent to day stock, so the number of tires in node day $i$ is zero. Each stock must send all tires to another stock or another day (through normal or quick services), so the number of tires in each stock is zero (except for last day stock, with must have $\ge 0$ tires).

The transportation among stocks has no limits in number and it is free. The reshaping from each stock to a specific day is unlimited, mas costs $Q$ for quick service, and $N$ for normal service.

**Aditional constraints are necessary**: The number of tires leaving day $i$ must be greater than or equal to $r_i$. This aditional constraint grants that each day will have its tires. So we add a minimum flow to the transportation at the end of each day.

In [5]:
# A reasonable case is when P > Q > N. For example, if P < Q, it wouldn't make sense to use Q at all.

P = 10
N = 5
Q = 8
r = [2, 2, 2, 2]

m = Model(with_optimizer(Cbc.Optimizer, logLevel = 0))

# Number of tires purchased
@variable(m, p[1:4] >= 0, Int)

# Flow of tires to stock
@variable(m, s[1:4] >= 0, Int)

# Flow among stocks
@variable(m, a[1:3] >= 0, Int)

# Quick services
@variable(m, q[1:3] >= 0, Int)

# Normal services
@variable(m, n[1:3] >= 0, Int)

# Maximize the costs
@objective(m, Min, P*sum(p) + Q*sum(q) + N*sum(n))

# Constraints through each day
@constraint(m, p[1] - s[1] == 0)
@constraint(m, p[2] + q[1] - s[2] == 0)
@constraint(m, p[3] + n[1] + q[2] - s[3] == 0)
@constraint(m, p[4] + n[2] + q[3] - s[4] == 0)

# Constraints through each stock
@constraint(m, s[1] - q[1] - n[1] - a[1] == 0)
@constraint(m, s[2] + a[1] - q[2] - n[2] - a[2] == 0)
@constraint(m, s[3] + a[2] - q[3] - a[3] == 0)
@constraint(m, s[4] + a[3] >= 0)

# Constraints of flow from each day
@constraint(m, s .>= r)

optimize!(m)

In [10]:
println("Cost: ", objective_value(m))
for i=1:4
    println("p_$i qty: ", value(p[i]))
end
for i=1:3
    println("q_$i qty: ", value(q[i]))
end
for i=1:2
    println("n_$i qty: ", value(n[i]))
end

Cost: 60.0
p_1 qty: 2.0
p_2 qty: 2.0
p_3 qty: 0.0
p_4 qty: 0.0
q_1 qty: 0.0
q_2 qty: 0.0
q_3 qty: 0.0
n_1 qty: 2.0
n_2 qty: 2.0


The program is very smart. We could think that, because $Q < P$, we would have only the first tires purchased, and then have the rest of the tires reshaped.
But it would require to use $Q$, however $N$ is cheaper.
So much cheaper that it is worth it to buy more tires to use only $N$.

**(b)** If the answer to **(a)** is no, how can the formulation be manipulated to become a network problem?
Draw the associated network.
(Hint.
Add a redundant constraint introducing a fictitious node.)

**Solução:** The anwer was yes.

**(c)** Assume that a tire may be reshaped only once.
How does the above model change?
Will it still be network problem?

**Solução:**

## Reference

```bibtex
@book{bradley1977applied,
  title={Applied mathematical programming},
  author={Bradley, Stephen P and Hax, Arnoldo C and Magnanti, Thomas L},
  year={1977},
  publisher={Addison-Wesley}
}
```